# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3) - Parte 1: Preparación de los datos

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn.cluster import KMeans, MeanShift
from sklearn import manifold, preprocessing, decomposition

# Funciones propias
from utils_limpieza import * 

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

---
# Preparación de Datos <span style="color:magenta">**(Paso 1)**</span>

Se repiten los pasos de curación realizados en la entrega anterior. A tener en cuenta:
* Algunos pasos que se hacían por separado, ahora se hacen todos juntos. 
* No se simplifica la variable `Deposito`, ya que se descarta desde el principio.
* lo del 10K
* No se hace el cálculo de promedio y varianza de los vectores finales.

1. **Cargamos el dataset crudo.**

In [ ]:
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas_raw = pd.read_csv(path)

2. **Eliminamos las siguientes variables:**
    * `INSCRIPCION`.
    * `CATEGORIA`.
    * `DESCRIPCION_CATEGORIA`.
    * `CATEGORIA (Ajustado)`.
    * `NOMBRE`.
    * `CM04`.
    * `DESC_TRATAMIENTO_FISCAL`.
    * `TRATAMIENTO_DIFERNCIAL`.
    * `TRATAMIENTO_FISCAL`.
    * `PORCENTAJE_COMISION_EMPRESA`.
    * `DEPOSITO`.

In [ ]:
ventas = limpiar_basic(ventas_raw, cols_drop=['INSCRIPCION', 'CATEGORIA', 'DESCRIPCION_CATEGORIA', 'CATEGORIA (Ajustado)', 
                                              'NOMBRE', 'CM04', 'DEPOSITO', 'DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_DIFERNCIAL', 
                                              'TRATAMIENTO_FISCAL', 'PORCENTAJE_COMISION_EMPRESA'])

3. **Renombramos como `Otros` las subcategorías que no tengan al menos 1 `MODELO` = 1.**

In [ ]:
ventas = renombrar_elementos(ventas, columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

4. **Eliminamos los registros que contienen las siguientes subcategorías:**
    * `Otros`.
    * `Instalación, Mantenimiento, Reparación, etc de productos varios` (a.k.a. `Mantenimiento`).

In [ ]:
ventas = ventas[ventas['SUB-CATEGORIA'] != 'Otros'].copy()
ventas = ventas[ventas['SUB-CATEGORIA'] != 'Instalación, Mantenimiento, Reparación, etc de productos varios'].copy()

5. **Creamos la variable `Fecha` y eliminamos las variables `MES` y `AÑO`.**

In [ ]:
ventas["Fecha"] = pd.to_datetime(ventas['MES'].astype(str) + '-' + ventas['AÑO'].astype(str), format='%m-%Y')
ventas = limpiar_basic(ventas, cols_drop=['MES', 'AÑO'])

6. **Anonimizamos la variable sensible `ID_VENDEDOR`.**

In [ ]:
ventas, _ = anonimizar(ventas, 'ID_VENDEDOR')

7. **Simplificamos el nombre de las variables.**

In [ ]:
with open("../references/tp2_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas.rename(columns = column_dict, inplace = True)

8. **Simplificamos las categorías en `Subrubro`.**

In [ ]:
with open("../references/tp2_subrubro_dict.json") as subrubro_dict_json:
    subrubro_dict = json.load(subrubro_dict_json)

ventas['Subrubro'] = ventas['Subrubro'].replace(subrubro_dict)

9. <span style="color:magenta">**(Este paso es nuevo)**</span> **Remplazamos con $0 a todas las ventas y comisiones que estén en el rango desde -$10.000 hasta $10.000.**

In [ ]:
ventas[['Ventas', 'Comision']].describe()

In [ ]:
cut = 10000
a = ventas[(ventas['Ventas'] < cut) & (ventas['Ventas'] > - cut)].copy()
b = a[a['Ventas']==0].copy()
print(f'Se trata de {a.shape[0]} registros. De los cuales {b.shape[0]} ya son idénticamente nulos.')
print(f'Es decir que tenemos {a.shape[0] - b.shape[0]} registros que no son nulos dentro de este rango (${-cut}, ${cut}).')
c = a[a['Ventas'] != 0].copy()
cpos = c[c['Ventas']>0]
cneg = c[c['Ventas']<0]
print(f'De los valores no nulos, hay {cpos.shape[0]} positivos y {cneg.shape[0]} negativos.')

sns.histplot(cpos['Ventas'])
sns.histplot(cneg['Ventas'])
plt.show()

In [ ]:
# Imputamos las ventas
ven = ventas['Ventas'].copy()
ven.mask((ven < cut) & (ven > - cut), inplace=True)
ven.fillna(0, inplace=True)
ventas['Ventas'] = ven

# Imputamos las comisiones
ven = ventas['Comision'].copy()
ven.mask((ven < cut) & (ven > - cut), inplace=True)
ven.fillna(0, inplace=True)
ventas['Comision'] = ven

10. **Eliminamos el efecto de la inflación.**

In [ ]:
precios_path = "../data/external/tp2_IPC_Indec.csv"

if exists(precios_path):
    print('Este archivo ya existe.')
else:
    print('Este archivo no existe: ¡Vamos a crearlo!')
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_06_23.xls"
    df = pd.read_excel(url, sheet_name="Índices IPC Cobertura Nacional", header=None, usecols="B:CA", skiprows=[0, 1, 2, 3, 4, 6, 7, 8], nrows=2)
    df = df.transpose()
    df.columns = ["Fecha", "INDICE"]
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df.to_csv(precios_path, index=False)

precios = pd.read_csv(precios_path)
precios.columns = ["Fecha", "INDICE"]

# Hay que asegurar que la variable clave tenga el mismo tipo en los dos dataframes
precios["Fecha"] = pd.to_datetime(precios["Fecha"])

ventas = ventas.merge(precios[["Fecha", "INDICE"]], on="Fecha", how="left")

In [ ]:
# Hacemos la imputación
indexar(ventas, 'Ventas')
indexar(ventas, 'Comision')

In [ ]:
# Se descartan las variables que no se usan
ventas = limpiar_basic(ventas, cols_drop=['Ventas', 'Comision', 'INDICE'])

# Se renombran las variables
ventas.rename(columns = {'Ventas_Real': 'Ventas', 
                         'Comision_Real': 'Comision'}, inplace = True)

# Se reacomodan las columnas
ventas = ventas[['ID', 'Omega', 'Subrubro', 'Fecha', 'Ventas', 'Comision', 'Modelo']]

ventas = ventas.sort_values(['Fecha', 'Ventas']).reset_index(drop=True)

11. **Pasar de ventas a vendedores**

    Para cada combinación posible de `Subrubro`, `ID` y `Fecha`, vamos a:
    * Sumar todos los valores de `Ventas`.
    * Sumar todos los valores de `Comision`.
    * Mantener los valores de `Modelo` y de `Omega`.

    Al hacer esto, se agregan en una única fila todas las observaciones que pertenezcan a un mismo vendedor en una dada fecha bajo un cierto subrubro.

In [ ]:
agregado = ventas.groupby(['Subrubro', 'ID', 'Fecha']).agg({
    'Ventas': 'sum',
    'Comision': 'sum',
    'Modelo': 'min',
    'Omega': 'min'
}).reset_index()

12. **Se eliminan los registros asociados a vendedores que tienen siempre `Ventas` nulas dentro de un mismo `Subrubro`.**

In [ ]:
agregado_no_nulo = agregado.groupby(['Subrubro', 'ID']).filter(lambda x: (x['Ventas'] != 0).any()).copy()

13. **Deberíamos tener 42 observaciones por par `ID`/`Subrubro`. Imputamos los meses faltantes con valor 0 en `Ventas` y en `Comision`.**

In [ ]:
# Distinguimos los valores originales de los imputados (`Dato_original`).
agregado_no_nulo['Dato_original'] = 1

# Contar las observaciones por combinación de ID y Subrubro
conteo_combinaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().reset_index(name='Conteo')

# Filtrar las combinaciones con menos de 42 observaciones
combinaciones_faltantes = conteo_combinaciones[conteo_combinaciones['Conteo'] < 42]

# Lista para almacenar las observaciones faltantes
observaciones_faltantes = []

# Iterar sobre las combinaciones faltantes
for _, combinacion in combinaciones_faltantes.iterrows():
    id_val = combinacion['ID']
    subrubro_val = combinacion['Subrubro']
    
    # Obtener fechas existentes y fechas faltantes
    fechas_existentes = agregado_no_nulo[(agregado_no_nulo['ID'] == id_val) & (agregado_no_nulo['Subrubro'] == subrubro_val)]['Fecha']
    fechas_faltantes = set(agregado_no_nulo['Fecha'].unique()) - set(fechas_existentes)
    
    # Agregar observaciones faltantes al DataFrame
    for fecha_faltante in fechas_faltantes:
        observacion = {
            'ID': id_val,
            'Subrubro': subrubro_val,
            'Fecha': fecha_faltante,
            'Ventas': 0,
            'Comision': 0,
        }
        observaciones_faltantes.append(observacion)

# Crear DataFrame con las observaciones faltantes
df_observaciones_faltantes = pd.DataFrame(observaciones_faltantes)

# Agregar observaciones faltantes al DataFrame agregado_no_nulo
agregado_limpio = pd.concat([agregado_no_nulo, df_observaciones_faltantes], ignore_index=True)

# Imputo los valores Omega y Modelo asociados a los nuevos meses adicionados. Es más eficiente agregarlo en una celda aparte y no en el for (antes demoraba 9 minutos, ahora menos de 1).
agregado_limpio['Omega'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Omega'].transform('max')
agregado_limpio['Modelo'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Modelo'].transform('max')

# Relleno los valores nulos de Dato_original con 0.
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].fillna(0)

# Convertir las columnas 'Omega', 'Modelo' y 'Dato_original' a tipo int64
agregado_limpio['Omega'] = agregado_limpio['Omega'].astype('int64')
agregado_limpio['Modelo'] = agregado_limpio['Modelo'].astype('int64')
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].astype('int64')

promedio_observaciones = agregado_limpio.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(f'Cade vendedor tiene asociados {promedio_observaciones} meses diferentes.')

14. **Ordenar luego de todo lo que se hizo.**

In [ ]:
# Dejar que primero esté Fecha. Sirve para el pivoteo que hacemos después.
registros_vendedores_abs = agregado_limpio.sort_values(['Fecha', 'Subrubro', 'ID']).reset_index(drop=True)

# Gurdamos el dataset
registros_vendedores_abs.to_csv('../data/interim/tp3_registros_vendedores_abs.csv', index=False)

15. **Reemplazar ceros por NaN**

    Queremos hacer el cálculo de la variación porcentual. Con el dataset como está actualmente (presencia de ceros), pueden generarse resultados que tienden a infinito. Para superar esta situación, se reemplazan todos los ceros por NaN.

In [ ]:
registros_vendedores_rel = registros_vendedores_abs.copy()
registros_vendedores_rel[['Ventas', 'Comision']] = registros_vendedores_rel[['Ventas', 'Comision']].replace({0:np.NaN})

16. **Cálculo de cambios porcentuales.**

    **NOTA:** La función pct_change devuelve en el rango normal, no en el porcentual

In [ ]:
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 4)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 4)

17. **Pivotear los datos.**

In [ ]:
pivotear = registros_vendedores_rel.copy()

# Extraer el mes y el año de la columna "Fecha"
pivotear["Fecha"] = pd.to_datetime(pivotear["Fecha"])
pivotear["Month"] = pivotear["Fecha"].dt.month
pivotear["Year"] = pivotear["Fecha"].dt.year

# Convertir "Month" a string con formato de dos cifras
pivotear["Month"] = pivotear["Month"].apply(lambda x: str(x).zfill(2))

# Convertir "Year" a string y quedarse con los últimos 2 dígitos
pivotear["Year"] = pivotear["Year"].apply(lambda x: str(x)[-2:])

# Crear la variable "Fecha2" que concatena "Year" y "Month"
pivotear["Fecha2"] = pivotear["Year"] + pivotear["Month"]

# Eliminar columnas
pivotear.drop(columns=['Fecha','Ventas','Comision','Dato_original','Month','Year'], inplace=True)

# Renombrar la columnas
pivotear.rename(columns={'Fecha2': 'Fecha', 'Y_pct_Ventas': 'Y_pct_Ven', 'Y_pct_Comision': 'Y_pct_Com', 'F_pct_Ventas': 'F_pct_Ven', 'F_pct_Comision': 'F_pct_Com'}, inplace=True)

In [ ]:
# Realizar el pivoteo y la agrupación
pivot_df = pivotear.pivot_table(index=["ID",'Subrubro','Omega','Modelo'], columns=['Fecha'], values=['Y_pct_Ven', 'Y_pct_Com', 'F_pct_Ven', 'F_pct_Com'])

# Generar los nombres de las columnas finales
columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

# Asignar los nuevos nombres de columnas
pivot_df.columns = columns

# Restablecer el índice para que "ID" vuelva a ser una columna
pivot_df = pivot_df.reset_index()

18. **Guaradamos todo lo hecho.**

In [ ]:
pivot_df.to_csv('../data/interim/tp3_vendedores_vector.csv', index=False)

---
# Análisis del dataset antes de hacer agregaciones y pivotearlo: veamos con qué nos quedamos ahora

## Análisis de variables categóricas: `Subrubro` vs `Modelo`

In [ ]:
checkear_unicidad(ventas, 'Subrubro', 'ID')
graficar_modelo(ventas, 'Subrubro')

## Análisis de variables numéricas

In [ ]:
ventas[['Ventas', 'Comision']].describe()

In [ ]:
sns.scatterplot(data=ventas[['Ventas', 'Comision']], x='Ventas', y='Comision')
plt.show()

# Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

for col in ['Omega', 'Subrubro', 'Modelo']:
    sns.scatterplot(data=ventas[['Ventas', 'Comision',col]], x='Ventas', y='Comision', hue=col)
    plt.show()

## Análisis de la serie temporal

En todos los subrurbos vemos que hay modelos y no modelo, incluso si exluimos las ventas nulas.

In [ ]:
g = sns.relplot(
    data=ventas,
    x="Fecha", y="Ventas", col="Subrubro", hue="Modelo",
    kind="line", linewidth=2, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=True,
    facet_kws=dict(sharey=False)
)

# Iterate over each subplot to customize further
for subrubro, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.5, .95, subrubro, transform=ax.transAxes)

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.xticks(rotation=45, ha='right')

g.set_titles("'Ventas (totales)'")
g.set_axis_labels("", "Ventas")    
#g.tight_layout()

plt.show()

In [ ]:
g = sns.relplot(
    data=ventas[ventas['Ventas'] != 0],
    x="Fecha", y="Ventas", col="Subrubro", hue="Modelo",
    kind="line", linewidth=2, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=True,
    facet_kws=dict(sharey=False)
)

# Iterate over each subplot to customize further
for subrubro, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.5, .95, subrubro, transform=ax.transAxes)

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.xticks(rotation=45, ha='right')

g.set_titles("'Ventas (no nulas)'")
g.set_axis_labels("", "Ventas")    
#g.tight_layout()

plt.show()

---
# Datasets pivoteados: tp2 vs tp3

Vamos a comparar el dataset pivoteado que generamos al final del tp2 con el dataset pivoteado que generamos ahora, cuya principal diferencia es el punto 9 del pretratamiento: remplazamos con $0 a todas las ventas y comisiones que estaban en el rango desde -$10.000 hasta $10.000.

In [ ]:
tp2 = pd.read_csv('../data/interim/tp2_vendedores_vector_resumen.csv')
tp2 = limpiar_basic(tp2, cols_drop=['Omega', 'F_ven_mean', 'F_ven_var', 
                                    'F_com_mean', 'F_com_var', 'Y_ven_mean', 
                                    'Y_ven_var', 'Y_com_mean', 'Y_com_var'])

tp3 = pd.read_csv('../data/interim/tp3_vendedores_vector.csv')
tp3 = limpiar_basic(tp3, cols_drop='Omega')

Las filas de estos datasets representaban un vendedor único, dado el subrubro. Esto implica que un mismo `ID` puede aparecer más de una vez, pero es porque ese vendedor vende en dos o más `Subrubros`.

In [ ]:
print(f'El del tp2 tenía {tp2.shape[0]} filas, mientras que el nuevo tiene {tp3.shape[0]}.')
print(f'Hay {tp2.shape[0] - tp3.shape[0]} menos.')

## Sin diferenciar por subrubro

En la siguiente figura se grafican las medias con sus respectivas desviaciones estándares para cada variable porcentual. Vemos que, al haber agregado este paso nuevo en el pretratamiento, los datos son muchísimo menos ruidosos.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(25, 20))

axs[0, 0].set_title('tp2', fontsize=20)
axs[0, 0].errorbar(range(len(tp2.iloc[:, 3:].mean())), tp2.iloc[:, 3:].mean(), yerr = tp2.iloc[:, 3:].std(), capsize = 8)
axs[0, 0].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
axs[0, 0].tick_params(axis='y', labelsize=20)

axs[0, 1].set_title('tp2 (zoom)', fontsize=20)
axs[0, 1].errorbar(range(len(tp2.iloc[:, 3:].mean())), tp2.iloc[:, 3:].mean(), yerr = tp2.iloc[:, 3:].std(), capsize = 8)
axs[0, 1].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
axs[0, 1].set_ylim(-1E3, 1E3)
axs[0, 1].tick_params(axis='y', labelsize=20)

axs[1, 0].set_title('tp3', fontsize=20)
axs[1, 0].errorbar(range(len(tp3.iloc[:, 3:].mean())), tp3.iloc[:, 3:].mean(), yerr = tp3.iloc[:, 3:].std(), capsize = 8)
axs[1, 0].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
axs[1, 0].tick_params(axis='y', labelsize=20)

axs[1, 1].set_title('tp3 (zoom)', fontsize=20)
axs[1, 1].errorbar(range(len(tp3.iloc[:, 3:].mean())), tp3.iloc[:, 3:].mean(), yerr = tp3.iloc[:, 3:].std(), capsize = 8)
axs[1, 1].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
axs[1, 1].set_ylim(-50, 50)
axs[1, 1].tick_params(axis='y', labelsize=20)

plt.show()

Al comparar entre valores promedios, vemos que hay dos variables cuyo promedio ha bajado drásticamente. Para el resto, en general ha bajado en la mayoría de los casos, salvo algunos donde ha subido.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(25, 10))

axs[0].plot(range(len(tp3.iloc[:, 3:].mean())), tp3.iloc[:, 3:].mean() - tp2.iloc[:, 3:].mean())
axs[0].set_ylabel(r'Promedio tp3 - promedio tp2', fontsize=20)
axs[0].tick_params(axis='y', labelsize=20)

axs[1].plot(range(len(tp3.iloc[:, 3:].mean())), tp3.iloc[:, 3:].mean() - tp2.iloc[:, 3:].mean())
axs[1].set_ylabel(r'Promedio tp3 - promedio tp2', fontsize=20)
axs[1].tick_params(axis='y', labelsize=20)
axs[1].set_ylim(-100, 100)

plt.show()

Veamos cuáles eran los órdenes de magnitud de valores más extremos a lo largo de todo el dataset. Antes las variaciones porcentuales cubrían 13 órdenes de magnitud, mientras que ahora ocupan 6.

In [ ]:
print('TP2:')
print(f'\t El mínimo era {np.min(tp2.iloc[:, 3:].min()):.2e}')
print(f'\t El máximo era {np.max(tp2.iloc[:, 3:].max()):.2e}')

print('TP3:')
print(f'\t El mínimo es {np.min(tp3.iloc[:, 3:].min()):.2e}')
print(f'\t El máximo es {np.max(tp3.iloc[:, 3:].max()):.2e}')

## Diferenciando por subrubro

Al desagregar por subrubro, se sigue manteniendo el hecho de que los datos son muchísimo menos ruidosos ahora. Sin embargo, se ve que algunos subrubros presentan una mayor variación entre el tp2 y tp3 que otros: 'Farmacia', 'Comb.', 'Vehiculos', 'Tabaco' y 'Comb. Reventa' presentan las variaciones más leves.

In [ ]:
for sr in tp2.Subrubro.unique():

    tp2sr = tp2[tp2['Subrubro'] == sr].copy()
    tp3sr = tp3[tp3['Subrubro'] == sr].copy()

    fig, axs = plt.subplots(1, 2, figsize=(25, 10))

    fig.suptitle(sr, fontsize=20)

    axs[0].set_title('tp2', fontsize=20)
    axs[0].errorbar(range(len(tp2sr.iloc[:, 3:].mean())), tp2sr.iloc[:, 3:].mean(), yerr = tp2sr.iloc[:, 3:].std(), capsize = 8)
    axs[0].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
    axs[0].tick_params(axis='y', labelsize=20)

    axs[1].set_title('tp3', fontsize=20)
    axs[1].errorbar(range(len(tp3sr.iloc[:, 3:].mean())), tp3sr.iloc[:, 3:].mean(), yerr = tp3sr.iloc[:, 3:].std(), capsize = 8)
    axs[1].set_ylabel(r'Promedio $\pm$ SD', fontsize=20)
    axs[1].tick_params(axis='y', labelsize=20)

    plt.show()

---
# Datos faltantes en el nuevo dataset pivoteado

## Sin distinguir por subrubro

Vemos que hay una gran cantidad de datos faltantes en todas las variables asociadas a la variación porcentual. Más aún: ninguna de estas columnas posee el 100% de los datos. Para cada grupo de variables (según sea comisión/ventas o anual/cuatrimestral) la cantidad de datos faltantes disminuye al progresar en el tiempo (lo cual tiene sentido).

Las variables asociadas a la comisión presentan más datos faltantes que aquellas asociadas a las ventas. Dentro de ventas, hay más datos en el caso cuatrimestral que en el caso anual.

**Nota:** En la gráfica aparecen, desde arriba hacia abajo, ventas anuales, comisión anuales, ventas cuatrimestrales y comisión cuatrimestrales. Dentro de cada grupo, aparecen desde el mes más reciente hacia el más antigüo.

In [ ]:
msno.bar(tp3.iloc[:, 3:], fontsize=12, color="tab:green", figsize=(24, 20))
plt.show()

In [ ]:
msno.bar(tp3[['F_pct_Ven_1905', 'F_pct_Ven_2206', 'F_pct_Com_1905', 'F_pct_Com_2206', 'Y_pct_Ven_2001', 'Y_pct_Ven_2206', 'Y_pct_Com_2001', 'Y_pct_Com_2206']], fontsize=12, color="tab:green", figsize=(12, 10))
plt.ylim(0.6, 1)
plt.show()

Por lo antes dicho, seguiremos el análisis con la ventas intercuatrimestrales.

In [ ]:
basics = ['ID', 'Subrubro', 'Modelo']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

tp3_Fven = tp3[basics+F_Ven].copy()

In [ ]:
msno.bar(tp3_Fven, fontsize=12, color="tab:green", figsize=(12, 10))
plt.ylim(0.8, 1)
plt.show()

## Diferenciando por subrubro

Hay de todo un poco. Algunos casos presentan menos de 80% de los datos. La mayoría sigue la tendencia de ir aumentando a medida que pasa el tiempo. Algunos son más "consistentes" que otros.

In [ ]:
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()

    plt.suptitle(sr)
    msno.bar(tp3sr, fontsize=12, color="tab:green", figsize=(10, 7))
    plt.ylim(0.8, 1)
    plt.show()

## Cantidad de vacíos por vendedor

De las 41 columnas, 3 son el `ID`, el `Subrubro` y `Modelo`. Las otras 38 corresponder a las variaciones porcentuales intercuatrimestrales. Si tomamos como aceptable un 10% de datos faltantes, podemos tolerar como máximo que haya 4 datos faltantes.

De los 4544 vendedores únicos, 3488 (el 77 %) no presentan ningún dato faltante. En el otro extremo, llegamos a tener vendedores con 37 datos faltantes, siendo que son 38 columnas. Acá hay algo raro, que debemos analizar.

In [ ]:
# Paso 1: Contar valores vacíos por fila
vacios_por_fila = tp3_Fven.isnull().sum(axis=1)

# Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
resumen = vacios_por_fila.value_counts().reset_index()
resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')

plt.figure(figsize=(10, 4))  # Tamaño del gráfico
plt.bar(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], color='blue', alpha=0.7)
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor')
plt.xticks(resumen['Cantidad de Valores Vacíos'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 200)

plt.show()

display(resumen)

Analicemos esto mismo, pero por subrubro. `Miscelaneo` presenta la mayor cantidad de vendedores con diferente cantidad de datos faltantes. En todos los casos hay algún vendedor con más de 30 datos faltantes.

In [ ]:
plt.figure(figsize=(10, 4))  # Tamaño del gráfico

for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()

    # Paso 1: Contar valores vacíos por fila
    vacios_por_fila = tp3sr.isnull().sum(axis=1)

    # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')

    plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 30)
plt.xlim(0, 40)

plt.legend(fontsize=7)

plt.show()

Vemos que en todos los subrubros tenemos modelo (por las dudas si nos había quedado alguno sin).

In [ ]:
print('Subrubro \t Cantidad de modelos')
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()
    
    print(f"{sr} \t {tp3sr['Modelo'].sum()}")

Vamos a repetir la gráfica de valores faltantes por subrubro, pero ahora tomando únicamente a los modelo. Sólo se grafican aquellos subrubros donde algún modelo posee al menos 1 dato  faltante. Estos subrubros resultaron ser: `Comb.`, `Supermercados`, `Comb. Ley`, `Vehiculos` y `Tabaco`. Los últimos 4 casos presentan modelos con una cantidad de datos faltantes muy por encima del valor tolerable.

In [ ]:
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)].copy()

    # Paso 1: Contar valores vacíos por fila
    vacios_por_fila = tp3sr.isnull().sum(axis=1)

    # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
    if len(resumen) > 1:
        plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xlim(-1, 40)

plt.legend(fontsize=7)

plt.show()

Vamos a identificar a estos modelos con datos faltantes y corroborar si no es que pertenecen a más de un subrubro. Esto explicaría el porque de que aparezcan con datos faltantes en algún rubro.

Tomamos las siguientes decisiones
+ Eliminar al vendedor 306 de `Supermercados` y `Tabaco`, ya que sólo tiene 2 datos al principio.
+ Los 12 NaN del vendedor 494 correspoden a que vendía sólo en `Comb.` y con el tiempo empezó a vender también en `Comb. Ley`. Por este motivo no se lo elimina, sino que se lo imputará de alguna manera.
+ El vendedor 496 presenta un único dato faltante en dos de tres subrubros. Se lo imputará de alguna manera.
+ El vendedor 599 presenta 3 datos faltantes en uno de sus subrubros. Se lo imputará de alguna manera.
+ El vendedor 798 presenta 27 datos faltantes en `Vehiculos` y entre los 11 valores que no son NaN, solo hay 4 que son no nulos. Se lo elimina de esta categoría.
+ El vendedor 891 presenta un único dato faltante en un subrubro. Se lo imputará de alguna manera.

In [ ]:
# Busco los ID que tienen datos faltantes
ids = []
for sr in ['Supermercados', 'Comb.', 'Comb. Ley', 'Vehiculos', 'Tabaco']:

    tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)]['ID'].copy()
    for k in tp3sr:
        ids.append(k)

ids = np.unique(np.array(ids))

for k in ids:
    tp3sr = tp3_Fven[(tp3_Fven['ID'] == k)].copy()
    vacios_por_fila = tp3sr.isnull().sum(axis=1)
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
    if len(resumen) > 1:
        print(f'\n\nVendedor {k}:')
        display(tp3sr)
        display(resumen)

# for sr in tp2.Subrubro.unique():

#     tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)].copy()

#     # Paso 1: Contar valores vacíos por fila
#     vacios_por_fila = tp3sr.isnull().sum(axis=1)

#     # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
#     resumen = vacios_por_fila.value_counts().reset_index()
#     resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
#     resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
#     if len(resumen) > 1:
#         plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
# plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
# plt.ylabel('Frecuencia')
# plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xlim(-1, 40)

# plt.legend(fontsize=7)

# plt.show()

In [ ]:
tp3_Fven[['ID']].value_counts()

In [ ]:
# msno.matrix(tp3.iloc[:, 3:], fontsize=12, color=[0.5,0,0], figsize=(6, 5))
# plt.show()

In [ ]:
# # Definir grupos de variables:

# basics = ['ID', 'Subrubro', 'Modelo']

# F_Com = ['F_pct_Com_1905', 'F_pct_Com_1906', 'F_pct_Com_1907', 'F_pct_Com_1908', 
#          'F_pct_Com_1909', 'F_pct_Com_1910', 'F_pct_Com_1911', 'F_pct_Com_1912',
#          'F_pct_Com_2001', 'F_pct_Com_2002', 'F_pct_Com_2003', 'F_pct_Com_2004',
#          'F_pct_Com_2005', 'F_pct_Com_2006', 'F_pct_Com_2007', 'F_pct_Com_2008',
#          'F_pct_Com_2009', 'F_pct_Com_2010', 'F_pct_Com_2011', 'F_pct_Com_2012',
#          'F_pct_Com_2101', 'F_pct_Com_2102', 'F_pct_Com_2103', 'F_pct_Com_2104',
#          'F_pct_Com_2105', 'F_pct_Com_2106', 'F_pct_Com_2107', 'F_pct_Com_2108',
#          'F_pct_Com_2109', 'F_pct_Com_2110', 'F_pct_Com_2111', 'F_pct_Com_2112',
#          'F_pct_Com_2201', 'F_pct_Com_2202', 'F_pct_Com_2203', 'F_pct_Com_2204',
#          'F_pct_Com_2205', 'F_pct_Com_2206']

# F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
#          'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
#          'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
#          'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
#          'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
#          'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
#          'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
#          'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
#          'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
#          'F_pct_Ven_2205', 'F_pct_Ven_2206']

# Y_Com = ['Y_pct_Com_2001', 'Y_pct_Com_2002', 'Y_pct_Com_2003', 'Y_pct_Com_2004',
#          'Y_pct_Com_2005', 'Y_pct_Com_2006', 'Y_pct_Com_2007', 'Y_pct_Com_2008',
#          'Y_pct_Com_2009', 'Y_pct_Com_2010', 'Y_pct_Com_2011', 'Y_pct_Com_2012',
#          'Y_pct_Com_2101', 'Y_pct_Com_2102', 'Y_pct_Com_2103', 'Y_pct_Com_2104',
#          'Y_pct_Com_2105', 'Y_pct_Com_2106', 'Y_pct_Com_2107', 'Y_pct_Com_2108',
#          'Y_pct_Com_2109', 'Y_pct_Com_2110', 'Y_pct_Com_2111', 'Y_pct_Com_2112',
#          'Y_pct_Com_2201', 'Y_pct_Com_2202', 'Y_pct_Com_2203', 'Y_pct_Com_2204',
#          'Y_pct_Com_2205', 'Y_pct_Com_2206']

# Y_Ven = ['Y_pct_Ven_2001', 'Y_pct_Ven_2002', 'Y_pct_Ven_2003', 'Y_pct_Ven_2004',
#          'Y_pct_Ven_2005', 'Y_pct_Ven_2006', 'Y_pct_Ven_2007', 'Y_pct_Ven_2008',
#          'Y_pct_Ven_2009', 'Y_pct_Ven_2010', 'Y_pct_Ven_2011', 'Y_pct_Ven_2012',
#          'Y_pct_Ven_2101', 'Y_pct_Ven_2102', 'Y_pct_Ven_2103', 'Y_pct_Ven_2104',
#          'Y_pct_Ven_2105', 'Y_pct_Ven_2106', 'Y_pct_Ven_2107', 'Y_pct_Ven_2108',
#          'Y_pct_Ven_2109', 'Y_pct_Ven_2110', 'Y_pct_Ven_2111', 'Y_pct_Ven_2112',
#          'Y_pct_Ven_2201', 'Y_pct_Ven_2202', 'Y_pct_Ven_2203', 'Y_pct_Ven_2204',
#          'Y_pct_Ven_2205', 'Y_pct_Ven_2206']

# Tratamiento de vacíos

In [21]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn.cluster import KMeans, MeanShift
from sklearn import manifold, preprocessing, decomposition

# Funciones propias
from utils_limpieza import * 

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
tp3 = pd.read_csv('../data/interim/tp3_vendedores_vector.csv')
tp3 = limpiar_basic(tp3, cols_drop='Omega')

basics = ['ID', 'Subrubro', 'Modelo']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

tp3_Fven = tp3[basics+F_Ven].copy()

tp3_Fven['Vacios'] = tp3_Fven[F_Ven].isnull().sum(axis=1)

tp3_Fven

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios
0,0,Miscelaneo,0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964,0
1,1,Miscelaneo,0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497,0
2,2,Miscelaneo,0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505,0
3,4,Gondola,0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411,0
4,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,3111,Gondola,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014144,0.018062,-0.615873,0.870533,0.075915,-0.274677,-0.003937,-0.669660,0.053046,-0.642653,1.232293,0.636704,26
4540,3112,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.193040,0.193040,0.066273,0.066273,-0.106256,33
4541,3115,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,33
4542,3116,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.328046,7.706838,0.355923,35


En primer lugar, vamos a definir una variable llamada `Tipo_subrubro` que determina si el rubro en cuestión es *Primario* o *Secundario*. Donde cada vendedor va a tener un único rubro *Primario* y ninguno, uno o más rubros *Secundario*.
* Los vendedores con un solo rubro, van a tener el valor *Primario* en todos los casos.
* Para los vendedores con varios rubros, se tendrá en cuenta la siguiente regla de decisión:
    * Es *Primario* el subrubro con menor cantidad de valores vacíos (en caso de empate define el que tenga el mayor volumen de ventas)
    * El resto de los subrubros son *Secundario*.

In [23]:
# Encuentra los ID que aparecen más de una vez
duplicados = tp3_Fven[tp3_Fven.duplicated(subset='ID', keep=False)].sort_values(by=['ID', 'Subrubro']).copy()
duplicados

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios
4,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15
5,5,Venta Agrop.,0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715,0
9,9,Gondola,0,0.388525,0.098840,-0.192983,-0.133713,-0.088902,-0.116629,-0.092348,-0.356984,-0.153109,-0.022541,0.250874,0.814449,0.206797,0.237294,-0.007669,0.095842,-0.000951,-0.095607,-0.357594,-0.419077,-0.354451,-0.021810,0.584902,0.562483,0.650172,0.209990,-0.114101,0.026171,0.062030,-0.206309,0.019305,-0.235023,-0.348156,0.009262,0.310032,0.532440,0.686219,0.309455,0
10,9,Miscelaneo,0,0.535088,0.265234,0.832065,0.758694,0.004668,-0.080061,-0.264132,-0.458490,-0.151339,-0.145511,0.272602,1.018970,0.427751,0.609042,0.523308,0.101789,0.192377,0.055249,-0.393323,-0.176648,-0.363825,-0.245195,0.392693,0.150084,0.600681,0.479044,0.323475,-0.015030,0.021831,-0.177876,-0.323229,-0.347285,-0.453486,0.025441,0.474933,0.878675,1.413028,0.323410,0
11,10,Gondola,0,0.200221,-0.197894,-0.062205,0.489982,-0.281609,1.060372,-0.160490,0.315285,-0.096965,-0.337745,0.149428,-0.202024,0.153262,0.083904,0.034534,0.346065,0.627711,0.040108,-0.035409,0.558030,-0.048211,0.895523,0.925332,-0.375125,-0.346909,-0.240041,-0.239307,0.491580,1.027842,0.464767,0.191646,-0.067515,-0.581453,-0.545048,-0.032085,0.035080,1.147715,0.659789,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4490,3021,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.254503,-0.213428,1.697443,-0.052266,0.363723,0.162221,-0.376160,0.523719,0.216521,0.015686,0.769485,0.304280,0.057211,0.346525,24
4510,3050,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.360796,2.218779,0.552417,1.550753,-0.085831,-0.447891,-0.579756,-0.468952,0.402273,-0.327296,1.246851,-3.066042,-0.548206,-0.446316,-0.682177,-1.356416,0.152404,-0.184728,0.234793,-0.686754,-0.339731,0.093720,-0.130941,2.830609,1.076739,0.280998,12
4511,3050,Vehiculos,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.681081,-0.933243,0.423052,-0.113843,-0.414127,9.487463,0.214132,-0.551569,0.274338,-0.235245,-0.006573,-2.114940,-0.678045,-0.053129,-0.430274,-1.763029,0.402565,-0.074908,-0.139891,-0.511805,-0.062050,-0.154242,-0.326164,1.550707,-0.115532,-0.132099,12
4524,3083,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.978119,1.152841,0.534941,0.557827,66.002490,-0.262019,-0.156983,-0.332620,-0.403294,29


Necesitamos recuperar la variable `Ventas`

In [24]:
path = '../data/interim/tp3_registros_vendedores_abs.csv'
montos = pd.read_csv(path)
montos = montos.sort_values(by=['ID', 'Subrubro']).copy()
montos

,Subrubro,ID,Fecha,Ventas,Comision,Modelo,Omega,Dato_original
2021,Miscelaneo,0,2019-01-01,61155.916146,0.00,0,1,1
6631,Miscelaneo,0,2019-02-01,0.000000,0.00,0,1,1
11241,Miscelaneo,0,2019-03-01,0.000000,0.00,0,1,1
15851,Miscelaneo,0,2019-04-01,96651.469361,0.00,0,1,1
20461,Miscelaneo,0,2019-05-01,0.000000,0.00,0,1,1
...,...,...,...,...,...,...,...,...
174262,Miscelaneo,3131,2022-02-01,0.000000,0.00,0,1,0
178872,Miscelaneo,3131,2022-03-01,0.000000,0.00,0,1,0
183482,Miscelaneo,3131,2022-04-01,163688.033337,0.00,0,1,1
188092,Miscelaneo,3131,2022-05-01,205529.543519,0.00,0,1,1


In [25]:
# Crear la nueva variable 'Montos' que contendrá la suma de las Ventas por Subrubro e ID
montos['Montos'] = montos.groupby(['ID', 'Subrubro'])['Ventas'].transform('sum')
montos = montos[['ID', 'Subrubro', 'Montos']].copy()
montos = montos.drop_duplicates(subset=['ID', 'Subrubro'], keep='first').copy()
montos

,ID,Subrubro,Montos
2021,0,Miscelaneo,1.580649e+08
2022,1,Miscelaneo,9.074331e+08
2023,2,Miscelaneo,3.190490e+08
1413,4,Gondola,5.409168e+08
0,5,Com. Varios,8.633400e+06
...,...,...,...
3691,3122,Miscelaneo,1.759413e+07
578,3123,Com. Varios,5.975803e+04
2020,3126,Gondola,3.627312e+07
4281,3127,Vehiculos,9.985510e+05


In [26]:
# Mergear la variable Montos al df duplicados
duplicados = duplicados.merge(montos[['ID', 'Subrubro', 'Montos']], on=['ID', 'Subrubro'], how='left')
duplicados

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios,Montos
0,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15,8.633400e+06
1,5,Venta Agrop.,0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715,0,1.386944e+09
2,9,Gondola,0,0.388525,0.098840,-0.192983,-0.133713,-0.088902,-0.116629,-0.092348,-0.356984,-0.153109,-0.022541,0.250874,0.814449,0.206797,0.237294,-0.007669,0.095842,-0.000951,-0.095607,-0.357594,-0.419077,-0.354451,-0.021810,0.584902,0.562483,0.650172,0.209990,-0.114101,0.026171,0.062030,-0.206309,0.019305,-0.235023,-0.348156,0.009262,0.310032,0.532440,0.686219,0.309455,0,1.696637e+09
3,9,Miscelaneo,0,0.535088,0.265234,0.832065,0.758694,0.004668,-0.080061,-0.264132,-0.458490,-0.151339,-0.145511,0.272602,1.018970,0.427751,0.609042,0.523308,0.101789,0.192377,0.055249,-0.393323,-0.176648,-0.363825,-0.245195,0.392693,0.150084,0.600681,0.479044,0.323475,-0.015030,0.021831,-0.177876,-0.323229,-0.347285,-0.453486,0.025441,0.474933,0.878675,1.413028,0.323410,0,2.361150e+08
4,10,Gondola,0,0.200221,-0.197894,-0.062205,0.489982,-0.281609,1.060372,-0.160490,0.315285,-0.096965,-0.337745,0.149428,-0.202024,0.153262,0.083904,0.034534,0.346065,0.627711,0.040108,-0.035409,0.558030,-0.048211,0.895523,0.925332,-0.375125,-0.346909,-0.240041,-0.239307,0.491580,1.027842,0.464767,0.191646,-0.067515,-0.581453,-0.545048,-0.032085,0.035080,1.147715,0.659789,0,8.265416e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,3021,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.254503,-0.213428,1.697443,-0.052266,0.363723,0.162221,-0.376160,0.523719,0.216521,0.015686,0.769485,0.304280,0.057211,0.346525,24,1.081710e+07
2766,3050,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.360796,2.218779,0.552417,1.550753,-0.085831,-0.447891,-0.579756,-0.468952,0.402273,-0.327296,1.246851,-3.066042,-0.548206,-0.446316,-0.682177,-1.356416,0.152404,-0.184728,0.234793,-0.686754,-0.339731,0.093720,-0.130941,2.830609,1.076739,0.280998,12,7.343619e+07
2767,3050,Vehiculos,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.681081,-0.933243,0.423052,-0.113843,-0.414127,9.487463,0.214132,-0.551569,0.274338,-0.235245,-0.006573,-2.114940,-0.678045,-0.053129,-0.430274,-1.763029,0.402565,-0.074908,-0.139891,-0.511805,-0.062050,-0.154242,-0.326164,1.550707,-0.115532,-0.132099,12,2.750507e+09
2768,3083,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.978119,1.152841,0.534941,0.557827,66.002490,-0.262019,-0.156983,-0.332620,-0.403294,29,1.000865e+07


In [27]:
def etiquetar_tipo_subrubro(df_grupo):
    # Si hay empate en la cantidad de Vacíos, seleccionamos el que tenga el menor Promedio
    min_vacios = df_grupo['Vacios'].min()
    max_montos = df_grupo[df_grupo['Vacios'] == min_vacios]['Montos'].max()
    
    # Identificar el Primario y Secundarios
    df_grupo['Tipo_subrubro'] = 'Secundario'
    df_grupo.loc[(df_grupo['Vacios'] == min_vacios) & (df_grupo['Montos'] == max_montos), 'Tipo_subrubro'] = 'Primario'
    
    return df_grupo

# Aplicar la función de etiquetado por grupo (ID)
duplicados = duplicados.groupby('ID').apply(etiquetar_tipo_subrubro)
duplicados.reset_index(drop=True, inplace=True)

duplicados

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios,Montos,Tipo_subrubro
0,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15,8.633400e+06,Secundario
1,5,Venta Agrop.,0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715,0,1.386944e+09,Primario
2,9,Gondola,0,0.388525,0.098840,-0.192983,-0.133713,-0.088902,-0.116629,-0.092348,-0.356984,-0.153109,-0.022541,0.250874,0.814449,0.206797,0.237294,-0.007669,0.095842,-0.000951,-0.095607,-0.357594,-0.419077,-0.354451,-0.021810,0.584902,0.562483,0.650172,0.209990,-0.114101,0.026171,0.062030,-0.206309,0.019305,-0.235023,-0.348156,0.009262,0.310032,0.532440,0.686219,0.309455,0,1.696637e+09,Primario
3,9,Miscelaneo,0,0.535088,0.265234,0.832065,0.758694,0.004668,-0.080061,-0.264132,-0.458490,-0.151339,-0.145511,0.272602,1.018970,0.427751,0.609042,0.523308,0.101789,0.192377,0.055249,-0.393323,-0.176648,-0.363825,-0.245195,0.392693,0.150084,0.600681,0.479044,0.323475,-0.015030,0.021831,-0.177876,-0.323229,-0.347285,-0.453486,0.025441,0.474933,0.878675,1.413028,0.323410,0,2.361150e+08,Secundario
4,10,Gondola,0,0.200221,-0.197894,-0.062205,0.489982,-0.281609,1.060372,-0.160490,0.315285,-0.096965,-0.337745,0.149428,-0.202024,0.153262,0.083904,0.034534,0.346065,0.627711,0.040108,-0.035409,0.558030,-0.048211,0.895523,0.925332,-0.375125,-0.346909,-0.240041,-0.239307,0.491580,1.027842,0.464767,0.191646,-0.067515,-0.581453,-0.545048,-0.032085,0.035080,1.147715,0.659789,0,8.265416e+08,Primario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,3021,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.254503,-0.213428,1.697443,-0.052266,0.363723,0.162221,-0.376160,0.523719,0.216521,0.015686,0.769485,0.304280,0.057211,0.346525,24,1.081710e+07,Primario
2766,3050,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.360796,2.218779,0.552417,1.550753,-0.085831,-0.447891,-0.579756,-0.468952,0.402273,-0.327296,1.246851,-3.066042,-0.548206,-0.446316,-0.682177,-1.356416,0.152404,-0.184728,0.234793,-0.686754,-0.339731,0.093720,-0.130941,2.830609,1.076739,0.280998,12,7.343619e+07,Secundario
2767,3050,Vehiculos,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.681081,-0.933243,0.423052,-0.113843,-0.414127,9.487463,0.214132,-0.551569,0.274338,-0.235245,-0.006573,-2.114940,-0.678045,-0.053129,-0.430274,-1.763029,0.402565,-0.074908,-0.139891,-0.511805,-0.062050,-0.154242,-0.326164,1.550707,-0.115532,-0.132099,12,2.750507e+09,Primario
2768,3083,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.978119,1.152841,0.534941,0.557827,66.002490,-0.262019,-0.156983,-0.332620,-0.403294,29,1.000865e+07,Primario


In [28]:
duplicados = duplicados[['ID', 'Subrubro', 'Tipo_subrubro']].copy()
duplicados

,ID,Subrubro,Tipo_subrubro
0,5,Com. Varios,Secundario
1,5,Venta Agrop.,Primario
2,9,Gondola,Primario
3,9,Miscelaneo,Secundario
4,10,Gondola,Primario
...,...,...,...
2765,3021,Venta Agrop.,Primario
2766,3050,Miscelaneo,Secundario
2767,3050,Vehiculos,Primario
2768,3083,Com. Varios,Primario


In [29]:
# Mergear la variable Tipo_subrubro al df tp3_Fven
tp3_Fven = tp3_Fven.merge(duplicados[['ID', 'Subrubro', 'Tipo_subrubro']], on=['ID', 'Subrubro'], how='left')

# Para los vacíos, asignar Primario
tp3_Fven['Tipo_subrubro'].fillna("Primario", inplace=True)

tp3_Fven

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios,Tipo_subrubro
0,0,Miscelaneo,0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964,0,Primario
1,1,Miscelaneo,0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497,0,Primario
2,2,Miscelaneo,0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505,0,Primario
3,4,Gondola,0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411,0,Primario
4,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15,Secundario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,3111,Gondola,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014144,0.018062,-0.615873,0.870533,0.075915,-0.274677,-0.003937,-0.669660,0.053046,-0.642653,1.232293,0.636704,26,Primario
4540,3112,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.193040,0.193040,0.066273,0.066273,-0.106256,33,Primario
4541,3115,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,33,Primario
4542,3116,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.328046,7.706838,0.355923,35,Primario


Una vez definida la variable `Tipo_subrubro`, vamos a tomar determinaciones respecto a qué hacer con los valores nulos.
Las reglas de decisión van a tener en cuenta a las variables `Modelo`, `Tipo_subrubro` y `Vacios`.

| Modelo | Tipo_subrubro | Vacíos         | Decisión       |
|--------|---------------|----------------|----------------|
| 1      | Primario      | 0              | No hacer nada  |
| 1      | Primario      | Hasta 20 (50%) | Imputar        |
| 1      | Primario      | Más de 20      | Tirar          |
| 1      | Secundario    | 0              | No hacer nada  |
| 1      | Secundario    | Hasta 10 (25%) | Imputar        |
| 1      | Secundario    | Más de 10      | Tirar          |
| 0      | Primario      | 0              | No hacer nada  |
| 0      | Primario      | Hasta 4 (10%)  | Imputar        |
| 0      | Primario      | Más de 4       | Tirar          |
| 0      | Secundario    | 0              | No hacer nada  |
| 0      | Secundario    | Hasta 2 (5%)   | Imputar        |
| 0      | Secundario    | Más de 2      | Tirar          |

In [30]:
# Crear una función para determinar la decisión
def determinar_decision(row):
    if row['Modelo'] == 1:
        if row['Tipo_subrubro'] == 'Primario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 20:
                return 'Imputar'
            else:
                return 'Tirar'
        elif row['Tipo_subrubro'] == 'Secundario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 10:
                return 'Imputar'
            else:
                return 'Tirar'
    elif row['Modelo'] == 0:
        if row['Tipo_subrubro'] == 'Primario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 4:
                return 'Imputar'
            else:
                return 'Tirar'
        elif row['Tipo_subrubro'] == 'Secundario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 2:
                return 'Imputar'
            else:
                return 'Tirar'
    return 'Desconocido'

In [31]:
# Aplicar la función para crear la columna "Decisión"
tp3_Fven['Decision'] = tp3_Fven.apply(determinar_decision, axis=1)
tp3_Fven

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206,Vacios,Tipo_subrubro,Decision
0,0,Miscelaneo,0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964,0,Primario,No hacer nada
1,1,Miscelaneo,0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497,0,Primario,No hacer nada
2,2,Miscelaneo,0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505,0,Primario,No hacer nada
3,4,Gondola,0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411,0,Primario,No hacer nada
4,5,Com. Varios,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450596,-0.237927,-0.237927,0.806316,-0.474648,-0.661039,-0.661039,1.797929,0.195350,20.298763,20.298763,1.580285,5.039614,0.000000,-0.834933,-0.834933,-0.834933,-0.834933,-0.908025,3.914401,3.914401,3.914401,52.431941,15,Secundario,Tirar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,3111,Gondola,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014144,0.018062,-0.615873,0.870533,0.075915,-0.274677,-0.003937,-0.669660,0.053046,-0.642653,1.232293,0.636704,26,Primario,Tirar
4540,3112,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.193040,0.193040,0.066273,0.066273,-0.106256,33,Primario,Tirar
4541,3115,Venta Agrop.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,33,Primario,Tirar
4542,3116,Miscelaneo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.328046,7.706838,0.355923,35,Primario,Tirar


## Aplicación de las decisiones

#### Tirar

In [32]:
tp3_Fven_limpio = tp3_Fven[tp3_Fven['Decision'] != 'Tirar'].copy()
tp3_Fven_limpio.drop(columns=['Decision', 'Vacios', 'Tipo_subrubro'], inplace=True)
tp3_Fven_limpio

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206
0,0,Miscelaneo,0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964
1,1,Miscelaneo,0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497
2,2,Miscelaneo,0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505
3,4,Gondola,0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411
5,5,Venta Agrop.,0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4455,2973,Farmacia,0,0.067191,0.168471,0.223234,0.204894,0.033863,0.270882,-0.058344,0.146829,0.023595,-0.238705,-0.431904,-0.858397,-0.406775,0.091405,0.722289,4.782569,0.626528,0.181471,-0.025004,0.234119,-0.021046,-0.208688,0.027877,-0.223529,-0.222467,-0.044828,0.074877,0.030315,0.414907,0.264124,-0.022700,0.329191,-0.122775,-0.156125,0.013776,-0.248115,0.026706,0.146889
4456,2974,Gondola,0,0.533815,0.271578,-0.197582,0.336976,2.397581,2.189153,0.475121,-0.718979,-0.921502,-0.857810,-0.536016,0.103845,0.165839,-0.214516,-0.323989,0.876081,7.582443,3.553055,3.426026,-0.907229,-0.893112,-0.856214,-0.746357,7.909287,0.784600,0.799278,0.244127,0.056499,1.523125,9.035588,2.327344,2.281358,-0.812570,-0.934353,-0.814483,-0.820076,0.155693,0.641247
4457,2975,Farmacia,0,-0.007857,0.163375,0.435761,-0.087601,-0.334698,0.472357,-0.349948,-0.175423,0.549314,-0.341120,0.118339,-0.052428,-0.316147,-0.060798,0.240392,0.807924,0.001004,0.133641,-0.525542,-0.576678,0.309949,-0.238732,1.034738,0.036236,0.117979,0.572682,0.022020,0.821329,0.115770,-0.349470,-0.258736,-0.274901,-0.496001,0.141077,-0.716947,-0.498321,-0.593563,-0.677739
4458,2977,Miscelaneo,0,-0.295017,-0.388342,-0.364785,-0.302340,0.413482,1.373845,0.137661,0.471830,0.000106,-0.190446,2.466280,0.422802,1.324961,0.788472,-0.328091,0.267426,-0.465998,0.073688,-0.552523,-0.44512

### Imputar con ceros

Para empezar, imputamos por 0 (luego decidir qué hacer)

In [33]:
# Imputar los valores faltantes con 0 en las columnas especificadas
tp3_Fven_limpio_ceros = tp3_Fven_limpio.copy()
tp3_Fven_limpio_ceros[F_Ven] = tp3_Fven_limpio_ceros[F_Ven].fillna(0).copy()
tp3_Fven_limpio_ceros

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206
0,0,Miscelaneo,0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964
1,1,Miscelaneo,0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497
2,2,Miscelaneo,0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505
3,4,Gondola,0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411
5,5,Venta Agrop.,0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4455,2973,Farmacia,0,0.067191,0.168471,0.223234,0.204894,0.033863,0.270882,-0.058344,0.146829,0.023595,-0.238705,-0.431904,-0.858397,-0.406775,0.091405,0.722289,4.782569,0.626528,0.181471,-0.025004,0.234119,-0.021046,-0.208688,0.027877,-0.223529,-0.222467,-0.044828,0.074877,0.030315,0.414907,0.264124,-0.022700,0.329191,-0.122775,-0.156125,0.013776,-0.248115,0.026706,0.146889
4456,2974,Gondola,0,0.533815,0.271578,-0.197582,0.336976,2.397581,2.189153,0.475121,-0.718979,-0.921502,-0.857810,-0.536016,0.103845,0.165839,-0.214516,-0.323989,0.876081,7.582443,3.553055,3.426026,-0.907229,-0.893112,-0.856214,-0.746357,7.909287,0.784600,0.799278,0.244127,0.056499,1.523125,9.035588,2.327344,2.281358,-0.812570,-0.934353,-0.814483,-0.820076,0.155693,0.641247
4457,2975,Farmacia,0,-0.007857,0.163375,0.435761,-0.087601,-0.334698,0.472357,-0.349948,-0.175423,0.549314,-0.341120,0.118339,-0.052428,-0.316147,-0.060798,0.240392,0.807924,0.001004,0.133641,-0.525542,-0.576678,0.309949,-0.238732,1.034738,0.036236,0.117979,0.572682,0.022020,0.821329,0.115770,-0.349470,-0.258736,-0.274901,-0.496001,0.141077,-0.716947,-0.498321,-0.593563,-0.677739
4458,2977,Miscelaneo,0,-0.295017,-0.388342,-0.364785,-0.302340,0.413482,1.373845,0.137661,0.471830,0.000106,-0.190446,2.466280,0.422802,1.324961,0.788472,-0.328091,0.267426,-0.465998,0.073688,-0.552523,-0.44512

### Comparación de datasets antes y después del tratamiento de nulos

In [34]:
# Crear los DataFrames "antes" y "ahora"
antes = tp3_Fven.pivot_table(index='Subrubro', columns='Modelo', aggfunc='size', fill_value=0)
ahora = tp3_Fven_limpio_ceros.pivot_table(index='Subrubro', columns='Modelo', aggfunc='size', fill_value=0)
diferencia = ahora - antes
porcentual = (ahora - antes)/antes*100

# Agregar nombres a los DataFrames
antes = antes.rename(columns=lambda x: (x, 'antes'))
ahora = ahora.rename(columns=lambda x: (x, 'después'))
diferencia = diferencia.rename(columns=lambda x: (x, 'diferencia'))
porcentual = porcentual.rename(columns=lambda x: (x, 'porcentual'))

# Concatenar los DataFrames con nombres
resultado = pd.concat([antes, ahora, diferencia, porcentual], axis=1)
resultado.columns = pd.MultiIndex.from_tuples(resultado.columns)
resultado.columns = resultado.columns.swaplevel(0, 1)  # Intercambiar el orden de los niveles de las columnas
resultado.columns.names = ['Tabla', 'Modelo']  # Cambiar el orden de los nombres de los niveles

# Mostrar el resultado
resultado

Tabla         antes     después     diferencia    porcentual      
Modelo            0   1       0   1          0  1          0     1
Subrubro                                                          
Com. Varios     555   4     392   4       -163  0 -29.369369   0.0
Comb.           274   6     226   6        -48  0 -17.518248   0.0
Comb. Ley       247   4     217   3        -30 -1 -12.145749 -25.0
Comb. Reventa    92   1      65   1        -27  0 -29.347826   0.0
Farmacia        198   4     185   4        -13  0  -6.565657   0.0
Gondola         595   3     478   3       -117  0 -19.663866   0.0
Miscelaneo     1637  20    1428  20       -209  0 -12.767257   0.0
Supermercados   203   4     169   3        -34 -1 -16.748768 -25.0
Tabaco           87   5      72   4        -15 -1 -17.241379 -20.0
Vehiculos       276   2     239   1        -37 -1 -13.405797 -50.0
Venta Agrop.    326   1     274   1        -52  0 -15.950920   0.0

In [35]:
tp3_Fven_limpio_ceros.to_csv('../data/interim/tp3_Fven_limpio_ceros.csv', index=False)

#### Imputar con KNN

In [36]:
tp3_Fven_limpio.columns

Index(['ID', 'Subrubro', 'Modelo', 'F_pct_Ven_1905', 'F_pct_Ven_1906',
       'F_pct_Ven_1907', 'F_pct_Ven_1908', 'F_pct_Ven_1909', 'F_pct_Ven_1910',
       'F_pct_Ven_1911', 'F_pct_Ven_1912', 'F_pct_Ven_2001', 'F_pct_Ven_2002',
       'F_pct_Ven_2003', 'F_pct_Ven_2004', 'F_pct_Ven_2005', 'F_pct_Ven_2006',
       'F_pct_Ven_2007', 'F_pct_Ven_2008', 'F_pct_Ven_2009', 'F_pct_Ven_2010',
       'F_pct_Ven_2011', 'F_pct_Ven_2012', 'F_pct_Ven_2101', 'F_pct_Ven_2102',
       'F_pct_Ven_2103', 'F_pct_Ven_2104', 'F_pct_Ven_2105', 'F_pct_Ven_2106',
       'F_pct_Ven_2107', 'F_pct_Ven_2108', 'F_pct_Ven_2109', 'F_pct_Ven_2110',
       'F_pct_Ven_2111', 'F_pct_Ven_2112', 'F_pct_Ven_2201', 'F_pct_Ven_2202',
       'F_pct_Ven_2203', 'F_pct_Ven_2204', 'F_pct_Ven_2205', 'F_pct_Ven_2206'],
      dtype='object')

In [37]:
# transponemos para imputar, porque knn funciona asignando el valor en función de las otras columnas. 
# Acá como cada feature en el mismo usuario en otro momento del tiempo los transpuse para que las columnas 
# sean otros usuarios en el mismo momento del tiempo.


X = tp3_Fven_limpio.iloc[:, 2:41].transpose() 
X.head(10)

,0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,47,49,50,51,53,54,55,56,58,59,60,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,83,84,85,...,4371,4372,4373,4374,4375,4377,4378,4379,4380,4381,4382,4384,4385,4387,4388,4389,4390,4392,4393,4394,4395,4397,4398,4399,4400,4402,4403,4405,4406,4407,4408,4409,4410,4411,4412,4413,4414,4415,4416,4417,4421,4423,4424,4425,4426,4427,4428,4429,4430,4431,4432,4433,4434,4435,4436,4437,4438,4439,4441,4442,4443,4444,4445,4447,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458,4459
Modelo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
F_pct_Ven_1905,0.580411,0.560242,0.012719,0.193896,1.728359,-0.144834,0.045961,1.516343,0.388525,0.535088,0.200221,0.184683,-0.748937,-0.201446,0.046896,0.506764,0.116311,-0.223867,0.209976,-0.008682,0.213298,0.133717,-0.006181,-0.469208,0.123540,-0.027324,0.008526,-0.053638,0.172718,-0.163838,0.712850,0.133330,-0.051598,0.097971,-0.551423,-0.038506,-0.147788,-0.239718,0.196408,0.320828,1.136220,-0.063100,NaN,0.591942,-0.566198,2.714597,0.016329,0.039437,-0.327536,0.295988,0.438401,0.589237,4.351189,0.635449,1.055873,-0.472567,-0.459485,0.570929,0.570929,-0.396991,1.447677,-0.095573,0.121924,0.174487,-0.215112,-0.239765,0.130734,-0.078840,-0.236461,-0.195561,0.818807,-0.145603,NaN,0.935693,0.513565,...,-0.013515,NaN,-0.646464,-0.003517,NaN,-0.843119,-0.741288,-0.661086,0.149072,0.268069,0.427806,-0.970815,0.244552,-0.305620,-0.390601,-0.193559,69.598795,-0.956955,1.337663,3.363683,0.270179,0.021470,10.179231,NaN,1.856975,0.633046,6.498275,0.035951,3.828096,-0.311377,-0.019837,-0.019837,-0.274292,0.083687,-0.208521,-0.352382,1.290054,0.371326,NaN,-0.051566,0.391842,1.980761,0.248662,0.906366,-0.382708,0.600323,-0.242873,5.791423,-0.065511,1.133562,NaN,-0.251411,-0.303496,-0.263885,-0.486297,-0.147815,0.641004,0.003674,-0.024976,0.129724,0.166744,-0.051571,-0.266767,24.668368,0.088910,-0.275451,-0.609126,-0.211553,-0.431445,0.335490,0.067191,0.533815,-0.007857,-0.295017,NaN
F_pct_Ven_1906,0.580411,0.592999,-0.010125,0.072714,4.203790,-0.221394,-0.080762,1.124226,0.098840,0.265234,-0.197894,-0.319247,-0.303047,-0.226079,0.259196,-0.131069,0.123820,0.111707,0.186090,0.042154,0.467439,0.083752,-0.968275,2.565795,0.145958,0.456453,-0.017435,0.535581,0.456529,-0.144372,-0.014358,0.622308,-0.512165,0.241779,-0.821085,0.072843,-0.346999,-0.126461,-0.215525,0.301877,-0.880814,0.396336,60.617121,0.253224,-0.494480,1.253544,0.297880,0.089730,0.044980,0.308967,0.028438,-0.029516,-0.457430,-0.691215,-0.445328,-0.438248,-0.085312,0.402489,0.402489,-0.248199,0.362271,-0.140277,0.881690,-0.219970,-0.246665,-0.118154,-0.151564,-0.3

In [38]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputer.fit(X)
X_imputed_arr = imputer.transform(X)

X_imputed_df_trans = pd.DataFrame(X_imputed_arr, columns=X.columns)

# vuelve a transponer para volver a la forma original
X_imputed_df = X_imputed_df_trans.transpose()
X_imputed_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,0.0,0.580411,0.580411,0.580411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,452.547408,457.728829,457.728829,457.728829,-0.203215,-0.710101,-0.710101,-0.938750,-0.922251,-0.788721,-0.788721,0.000000,0.000000,0.000000,-0.954996,-0.954996,0.035706,1.598752,17.949262,17.949262,-0.011453,-0.661136,0.032632,-0.240964
1,0.0,0.560242,0.592999,0.011296,-0.266983,-0.441363,-0.430089,-0.167779,1.110347,0.185458,0.176078,-0.489700,-0.664758,-0.894072,0.009338,0.879161,0.026533,4.183454,-0.535172,-0.373324,1.276247,0.168231,0.334280,0.487384,-0.320647,0.452886,0.495895,-0.136233,-0.266387,-0.202242,-0.348981,-0.031099,0.900433,-0.155532,0.492585,0.227850,-0.151318,1.190775,0.390497
2,0.0,0.012719,-0.010125,0.325712,0.167671,-0.260562,0.173718,-0.134293,0.477774,0.678065,0.010735,-0.388949,-0.983314,-0.636293,-0.263252,1.073850,57.508963,4.130725,3.139416,0.992096,0.655232,-0.302983,-0.683443,-0.052528,-0.247789,-0.133563,0.758891,-0.351847,-0.026916,0.240366,0.611252,-0.073709,-0.088358,-0.378793,-0.386948,-0.409333,-0.361089,0.010926,-0.265505
3,0.0,0.193896,0.072714,-0.055670,0.027243,-0.173982,0.008415,0.003202,0.050649,-0.078214,-0.414493,-0.069101,-0.136931,0.107958,0.343828,-0.020965,-0.037958,0.129644,0.456897,0.410191,0.651948,0.073848,0.034325,-0.022338,-0.118265,0.020972,-0.183847,0.038077,-0.020166,0.066091,0.140059,-0.115213,-0.036514,-0.431350,-0.373287,-0.174904,-0.235093,-0.004616,0.187411
5,0.0,1.728359,4.203790,-0.580229,0.168523,-0.796961,0.110348,1.634003,3.817642,0.554519,-0.707520,-0.567563,-0.398155,3.966332,4.059159,2.752705,-0.469371,-0.682299,-0.432133,-0.485836,0.373180,-0.416755,-0.416958,-0.112452,-0.634694,1.578138,1.100276,1.370289,4.154978,0.420448,0.309896,-0.485097,0.387735,-0.355202,-0.817434,-0.023355,-0.205820,-0.243716,10.140715


In [39]:
X_imputed = tp3_Fven_limpio.copy()
# inserto el df imputado en la misma forma de df del que se partió para tener las otras features categóricas (Subrubro, principalmente)
X_imputed.iloc[:, 2:41] = X_imputed_df

X_imputed.tail(10)

,ID,Subrubro,Modelo,F_pct_Ven_1905,F_pct_Ven_1906,F_pct_Ven_1907,F_pct_Ven_1908,F_pct_Ven_1909,F_pct_Ven_1910,F_pct_Ven_1911,F_pct_Ven_1912,F_pct_Ven_2001,F_pct_Ven_2002,F_pct_Ven_2003,F_pct_Ven_2004,F_pct_Ven_2005,F_pct_Ven_2006,F_pct_Ven_2007,F_pct_Ven_2008,F_pct_Ven_2009,F_pct_Ven_2010,F_pct_Ven_2011,F_pct_Ven_2012,F_pct_Ven_2101,F_pct_Ven_2102,F_pct_Ven_2103,F_pct_Ven_2104,F_pct_Ven_2105,F_pct_Ven_2106,F_pct_Ven_2107,F_pct_Ven_2108,F_pct_Ven_2109,F_pct_Ven_2110,F_pct_Ven_2111,F_pct_Ven_2112,F_pct_Ven_2201,F_pct_Ven_2202,F_pct_Ven_2203,F_pct_Ven_2204,F_pct_Ven_2205,F_pct_Ven_2206
4450,2970,Com. Varios,0.0,-0.275451,1.716239,0.349714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.342164,-0.342164,-0.342164,-0.342164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.596386,0.596386,-0.311568,-0.311568,-0.568756
4451,2970,Miscelaneo,0.0,-0.609126,-0.323665,-0.093767,-0.549570,-0.240108,0.525303,-0.030686,3.284666,0.724209,0.301607,-0.430918,-0.850913,-0.500671,-0.599464,0.289942,0.255968,0.152469,1.733275,0.723824,3.078834,0.283767,-0.483792,0.236341,-0.969500,-0.345658,-0.190690,-0.290411,11.158947,1.068875,0.418284,0.039135,-0.086452,-0.203970,-0.155047,0.125192,1.331043,0.220146,-0.017005
4452,2970,Vehiculos,0.0,-0.211553,0.074928,0.567465,-0.637632,-0.081356,-0.405413,-0.775619,2.653674,-0.580951,-0.016013,1.129969,-0.898152,1.611134,0.123615,-0.232938,-0.104326,-0.462338,1.131174,0.782971,5.152940,-0.309713,-0.898741,-0.045567,-0.481249,-0.547629,5.481232,-0.156138,0.267499,3.085220,-0.593170,-0.140725,-0.293590,-0.346460,0.243123,0.201740,2.227088,0.810657,0.621442
4453,2971,Miscelaneo,0.0,-0.431445,-0.115502,0.561268,0.333443,0.129741,0.359779,-0.117664,0.546344,1.321183,1.269858,-0.510467,-0.700503,-0.593200,-0.694735,0.046226,0.727891,-0.243145,-0.113558,0.522331,0.683918,1.346357,0.684051,0.461514,-0.290452,-0.388530,0.872109,0.615115,0.167305,0.169301,-0.485682,-0.315006,0.974378,0.921863,0.146987,-0.039839,-0.413393,-0.488175,-0.447525
4454,2971,Vehiculos,0.0,0.335490,-0.416041,-0.146735,-0.306323,-0.117453,1.009349,-0.037202,1.341723,0.119814,-0.350934,-0.017428,-0.036167,1.158909,1.093757,0.257096,0.451218,-0.480303,-0.470595,-0.078634,-0.387481,0.072293,0.368682,0.460858,0.552264,0.097162,-0.078511,-0.150954,0.007565,-0.173700,-0.102486,0.132048,-0.174317,0.030656,0.049108,0.107121,-0.048750,0.215126,0.407094
4455,2973,Farmacia,0.0,0.067191,0.168471,0.223234,0.204894,0.033863,0.270882,-0.058344,0.146829,0.023595,-0.238705,-0.431904,-0.858397,-0.406775,0.091405,0.722289,4.782569,0.626528,0.181471,-0.025004,0.234119,-0.021046,-0.208688,0.027877,-0.223529,-0.222467,-0.044828,0.074877,0.030315,0.414907,0.264124,-0.022700,0.329191,-0.122775,-0.156125,0.013776,-0.248115,0.026706,0.146889
4456,2974,Gondola,0.0,0.533815,0.271578,-0.197582,0.336976,2.397581,2.189153,0.475121,-0.718979,-0.921502,-0.857810,-0.536016,0.103845,0.165839,-0.214516,-0.323989,0.876081,7.582443,3.553055,3.426026,-0.907229,-0.893112,-0.856214,-0.746357,7.909287,0.784600,0.799278,0.244127,0.056499,1.523125,9.035588,2.327344,2.281358,-0.812570,-0.934353,-0.814483,-0.820076,0.155693,0.641247
4457,2975,Farmacia,0.0,-0.007857,0.163375,0.435761,-0.087601,-0.334698,0.472357,-0.349948,-0.175423,0.549314,-0.341120,0.118339,-0.052428,-0.316147,-0.060798,0.240392,0.807924,0.001004,0.133641,-0.525542,-0.576678,0.309949,-0.238732,1.034738,0.036236,0.117979,0.572682,0.022020,0.821329,0.115770,-0.349470,-0.258736,-0.274901,-0.496001,0.141077,-0.716947,-0.498321,-0.593563,-0.677739
4458,2977,Miscelaneo,0.0,-0.295017,-0.388342,-0.364785,-0.302340,0.413482,1.373845,0.137661,0.471830,0.000106,-0.190446,2.466280,0.422802,1.324961,0.788472,-0.328091,0.267426,-0.465998,0.073688,-0.552523,-0.445123,-0.031214,-0.596740,0.544067,-0.144223,-0.130646,1.454119,0.835476,0.888795,0.014053,-0.233204,-0.622248,-0.405013,-0.595008,-0.333223